# XOR-SAT Solver to Max Cut Problem

This Jupyter Notebook demonstrates an implementation of an XOR-SAT solver applied to the Max Cut problem. The Max Cut problem is a well-known problem in computer science and combinatorial optimization, where the goal is to partition the vertices of a graph into two disjoint subsets such that the number of edges between the subsets is maximized.

## Overview

The notebook is structured as follows:

1. **Library Imports**: Necessary libraries such as `numpy`, `math`, `matplotlib`, and others are imported for numerical computations and plotting.

2. **Function Definitions**: Several utility functions are defined, including:
    - `ReLU`: Rectified Linear Unit function.
    - `MP`: Message Passing function.
    - `Gradient`: Gradient computation function.
    - `rotation_to_bit`, `float_to_bit`, `bit_to_sign`: Functions for bit manipulation.
    - `temp_annealing`, `optimal_annealing`: Functions for simulated annealing.
    - `A_scaling`: Function for scaling.
    - `cosine_scheduling`: Function for cosine scheduling.
    - `manhattan_distance`: Function to compute Manhattan distance.
    - `gdflip`: Function to flip gradient direction.
    - `batch_norm`: Function for batch normalization.

3. **Variable Initialization**: Initialization of variables such as the number of variables (`n`), codeword, clauses (`H`), and other parameters required for the algorithm.

4. **Algorithm Implementation**: The main algorithm for solving the XOR-SAT problem and applying it to the Max Cut problem. This includes:
    - Hyperparameter setup.
    - Simulated annealing parameters.
    - Iterative process to find the optimal solution.
    - Recording and updating the best solution found during iterations.

5. **Results**: The final results are printed, including the total number of variables, number of clauses, number of iterations, and the maximum number of clauses satisfied during the iterations.

## Usage

To use this notebook, simply run each cell in sequence. The notebook will initialize the necessary variables, define the required functions, and execute the algorithm to solve the XOR-SAT problem applied to the Max Cut problem.



In [1]:
# Library import
import numpy as np
import math
import matplotlib.pyplot as plt
import time
import random
import math
np.random.seed(int(time.time()))

In [2]:
# function definition
def ReLU(a,b):
    return (a-b) * (a-b > 0)

ReLU = np.vectorize(ReLU)

def MP(z, tau):
    zeta = 0
    tau_p = sum(ReLU(z, zeta))
    # print(z)
    # print(tau)
    iteration = 0
    while(abs(tau_p-tau) > 1e-3) and iteration < 100:
        # print(abs(tau_p-tau))
        zeta += (tau_p-tau)/len(z)
        tau_p = sum(ReLU(z, zeta))
        iteration += 1
    # print("not out")
    return [z, zeta]


def Gradient(h, z_s, z_u, zeta_s, zeta_u, tau, A):

    return h*(ReLU(z_s, zeta_s)-ReLU(z_u, zeta_u))/((tau*A)+1)

Gradient = np.vectorize(Gradient)

def rotation_to_bit(x):
    return int((1-x)/2)

rotation_to_bit = np.vectorize(rotation_to_bit)

def float_to_bit(x):
    if(x >= 0):
        return 0
    else:
        return 1

float_to_bit = np.vectorize(float_to_bit)
    
def bit_to_sign(x):
    if(int(x) == 0):
        return 1
    else:
        return -1

bit_to_sign = np.vectorize(bit_to_sign)

def temp_annealing(iteration, max_iteration, init_temp, end_temp, energy):
    
    temp = init_temp*((end_temp/init_temp)**(iteration/max_iteration))
    
    return np.exp((-1*energy)/temp)

def optimal_annealing(t, c, init_temp, energy):
    # temp = init_temp/(math.log10(1+(t+1)/math.log2(c)))
    temp = (init_temp-c+10)/((math.log1p(t/1e4+1)))
    # return math.exp((-1)*(energy)/temp)
    prob = math.exp((-1)*(math.log1p(energy*1e4)/temp))
    if prob > 1:
        return 0.99
    elif prob < 0:
        return 0.01
    else: 
        return prob

def A_scaling(h, z_s, z_u, zeta_s, zeta_u):

    return h*((1 if z_s > zeta_s else 0.) + (1 if z_u > zeta_u else 0.))

A_scaling = np.vectorize(A_scaling)


def cosine_scheduling(mag, t):
    return mag/2*math.cos(t*math.pi/10)+mag/2

def manhattan_distance(a,b):
    return sum(np.logical_xor(a,b))

def gdflip(d,q,theta):
    if q >= theta:
        return d
    else:
        return -d

gdflip = np.vectorize(gdflip)

def batch_norm(a):
    average = np.average(a)
    std = np.std(a)
    return (a-average)/std

In [6]:
# Variable initialization 
n = 100

codeword = np.random.randint(0, 2, n)

m = int(n*10)

H=np.array([])

for i in range(m):
    x1 = np.random.randint(0,n)
    x2 = np.random.randint(0,n)
    if np.logical_xor(codeword[x1], codeword[x2]) == 1:
        new_clause = np.zeros(n)
        new_clause[x1] = 1
        new_clause[x2] = 1
        H = np.append(H, new_clause, axis=0)

H = np.reshape(H, (int(len(H)/n), n))
m = len(H)
print("Number of Clauses: ", len(H))

max_codeword = np.random.randint(0,2,n)

d = 1-2*max_codeword
r = np.random.uniform(-1, 1, n)
q = np.log10(np.abs(np.tanh(r)))*d

Number of Clauses:  502


In [7]:
# Algorithm

#hyper parameter

tau = m # MP constraint constant
theta = -2.1 # decision threshold
eta = 1.5 #learning rate
epsilon = 1e-3
beta = 0.9

#sitmulated annealing parameter
c = n
init_temp = m
end_temp = 0.01
init_codeword = d

grad_update = []

space = 2


#recording the codeword that satisfied the most clauses
max_clause = 0


#recording increasing decreasing
p_s_last = 0
c_batch = m
v_batch = n


#iteration
iteration = 0

#record variable

flip_grad = 0
flip_stimulated = 0
start_codeword = []
end_codeword = []
Q = np.array([])



flip_yet = False

q_last = q

max_d = d
    
while True:

    clauses_status = (1-np.prod(np.power(d, H), axis=1))/2
    num_satisfied = int(np.sum(clauses_status))
    
    if iteration%int(n/space) == 0:
        start_codeword.append((1-d)/2)
    elif iteration%int(n/space) == int(n/space) - 1:
        end_codeword.append((1-d)/2)

    if(num_satisfied == m):
        max_clause = num_satisfied
        print("original codeword: ", codeword)
        print("finish finding: ", (1-d)/2)
        print("difference: ", np.sum(np.abs(codeword-(1-d)/2)))
        break

    if(num_satisfied > max_clause):
        max_clause = num_satisfied
        max_d = d
        space += 0.05
        print("local max:", max_clause, " || ", iteration, " || ", optimal_annealing(iteration, max_clause, init_temp, num_satisfied))#, " || ", len(v_list))

    c_start = (iteration%int(m/c_batch))*c_batch+np.random.randint(0,m-int(m/c_batch)*c_batch+1)
    c_end = c_start + c_batch

    v_start = (iteration%int(n/v_batch))*v_batch
    v_end = v_start + v_batch

    v_list = np.random.permutation(n)[:int(100)]

    z = np.dot(H[c_start:c_end,v_list],q[v_list])
    z_s = z - np.multiply((1-clauses_status[c_start:c_end]), z+4)
    z_u = z - np.multiply((clauses_status[c_start:c_end]), z+4)
    
    [z_s, zeta_s] = MP(z_s, tau) 
    [z_u, zeta_u] = MP(z_u, tau) 

    d = gdflip(d,q,theta)

    A = np.sum(np.reshape([A_scaling(H[:,i], z_s, z_u, zeta_s, zeta_u) for i in v_list],(c_batch, len(v_list))), axis=0)

    HA = np.dot(np.multiply(H[c_start:c_end,v_list], A).transpose(), (z_s-z_u)/(m))
    q[v_list] = q[v_list] + eta*HA 
    Q = np.append(Q, q)

    idx = np.random.randint(0,n)
    if q[idx] < q_last[idx] and q[idx] > theta:
        d[idx] = -d[idx]

    q_last = q

    if iteration%int(n) == int(n)-1: 
        back_to_init = np.random.binomial(1, optimal_annealing(iteration, max_clause, init_temp, num_satisfied), n)
        q = batch_norm(q)
        q = q*(1-back_to_init)+back_to_init*np.log10(np.abs(q))*np.sign(np.random.uniform(-1, 1, n))
        

    iteration += 1

    


print("Total variables: ", n)
print("Number of Clauses: ", m)
print("Iterations: ", iteration)
print("-------------------------------------------------")
print("Max Clauses Satisfied (during the iterations): ", max_clause)
print("-------------------------------------------------")


local max: 231  ||  0  ||  0.9645012090665213
local max: 241  ||  3  ||  0.963103443207207
local max: 260  ||  4  ||  0.960174096500719
local max: 262  ||  5  ||  0.9598387350899719
local max: 270  ||  7  ||  0.9584505280933004
local max: 282  ||  9  ||  0.9561991949120736
local max: 292  ||  11  ||  0.954143199274588
local max: 293  ||  21  ||  0.953895971262878
local max: 294  ||  23  ||  0.9536726035117906
local max: 297  ||  25  ||  0.953003750685019
local max: 302  ||  27  ||  0.9518528310464991
local max: 304  ||  63  ||  0.951257730269532
local max: 305  ||  65  ||  0.9510107789613723
local max: 310  ||  79  ||  0.9497268477766756
local max: 311  ||  117  ||  0.9493390383316729
local max: 315  ||  126  ||  0.9482622096142211
local max: 316  ||  132  ||  0.9479729265606103
local max: 319  ||  134  ||  0.9471462064523931
local max: 331  ||  214  ||  0.9432974548837225
local max: 337  ||  216  ||  0.9413354342839101
local max: 338  ||  284  ||  0.9407241766776465
local max: 342  ||